In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,9723.91,0.01,0.12,0.01,0.01,-0.0,0.00
3,0.01,2643.47,-0.02,-0.02,-0.00,0.0,0.00
4,0.12,-0.02,545.99,-0.01,-0.00,-0.0,-0.00
5,0.01,-0.02,-0.01,81.28,-0.00,0.0,-0.00
6,0.01,-0.00,-0.00,-0.00,9.97,0.0,0.00
7,-0.00,0.00,-0.00,0.00,0.00,1.0,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.0,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00005,0.00001,0.00003,-0.00002,0.00002
3,0.00000,1.00000,-0.00002,-0.00004,-0.00003,0.00000,0.00001
4,0.00005,-0.00002,1.00000,-0.00003,-0.00003,-0.00002,-0.00000
5,0.00001,-0.00004,-0.00003,1.00000,-0.00002,0.00000,-0.00001
6,0.00003,-0.00003,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00002,0.00000,-0.00002,0.00000,0.00002,1.00000,0.00004
8,0.00002,0.00001,-0.00000,-0.00001,0.00001,0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18244722904526356

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.35387463e+07 9.70215659e+05 3.97295791e+04 8.26062349e+02
 1.10518868e+01 8.34300278e-02 3.89971127e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998769,-0.048716,-0.009217,-0.001368,-0.000167,-0.000017,-7.265826e-07
1,0.048262,0.997866,-0.043543,-0.006164,-0.000756,-0.000076,-3.437223e-06
2,0.011245,0.042777,0.998276,-0.038312,-0.004572,-0.000460,-2.100555e-05
3,0.002084,0.007683,0.037771,0.998379,-0.041618,-0.004094,-1.874205e-04
4,0.000339,0.001254,0.006069,0.041147,0.997843,-0.050724,-2.284886e-03
5,0.000052,0.000189,0.000916,0.006145,0.050350,0.997659,-4.584986e-02
6,0.000005,0.000017,0.000084,0.000563,0.004585,0.045674,9.989457e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00123081, 0.00213416, 0.00172412, 0.00162074, 0.00215719,
       0.00234075, 0.00105429])